In [12]:
import pandas as pd 

In [13]:
data = pd.read_csv("../data/extracted_data.csv",index_col=0)

In [14]:
data.head() 

,sidSecenek,cinsiyet,resimAdresi,urunGrubu,resimId
0,800567,BAYAN,https://showroom.aydinli.com.tr/2023Y/08/50265...,elbise,2
1,492805,ERKEK,https://resim.aydinli.com.tr/05/1/002/S5014643...,CEKET,3
2,733538,ERKEK,https://resim.aydinli.com.tr/02/1/082/S5024138...,sweat,2
3,618344,BAYAN,https://resim.aydinli.com.tr/02/2/011/S5020075...,tshirt,3
4,670262,ERKEK,https://resim.aydinli.com.tr/05/1/004/S5021961...,gomlek,1


In [15]:
data.cinsiyet.value_counts() 

ERKEK          18399
BAYAN          18057
ERKEK COCUK     8676
KIZ COCUK       6519
KIZ BEBEK         27
Name: cinsiyet, dtype: int64

In [16]:
data.urunGrubu.value_counts() 

CEKET        6000
sweat        6000
tshirt       6000
gomlek       6000
dis-giyim    6000
KAZAK        6000
pant         6000
elbise       4521
HIRKA        4101
ETEK         1056
Name: urunGrubu, dtype: int64

In [19]:
import cv2 
import os 
import requests 
import numpy as np 

def save_image_cv(url, scale_percent, path, size = None):
    resp = requests.get(url, stream=True).raw 
    
    image = np.asarray(bytearray(resp.read()), dtype="uint8")
    image = cv2.imdecode(image, cv2.IMREAD_COLOR)
    if image is None: 
        raise TypeError("None type") 
    
    if size == None: 
        width = int(image.shape[1] * scale_percent / 100)
        height = int(image.shape[0] * scale_percent / 100)
        dsize = (width, height)
    else: 
        dsize = size 
    
    output = cv2.resize(image, dsize)
    # output = cv2.cvtColor(output, cv2.COLOR_BGR2RGB) it is already in RGB Format  

    return cv2.imwrite(path, output)


def make_dataset_cv(dataset_name, datasource, iteration = None, set_name="train"):
    problems_rows = []
    iterator = 0 
    try:
        base = os.path.abspath(os.path.join(os.path.dirname(os.getcwd()), '.'))
        path = os.path.join('data', dataset_name)
        path = os.path.join(base, path)
        if not os.path.isdir(path):
            os.mkdir(path)
                
        folder_set = os.path.join(path, set_name) 
        os.mkdir(folder_set) 
    
        for img_index, img_row in datasource.iterrows():
                try:
                    ug_folder = os.path.join(folder_set, img_row["urunGrubu"]) 
                    if not os.path.isdir(ug_folder): 
                        os.mkdir(ug_folder) 
                    
                    if save_image_cv(img_row['resimAdresi'], 
                                     75, 
                                     os.path.join(ug_folder, str(img_index) + '.png'), 
                                     size=(720, 720)
                                     ) == False:
                        problems_rows.append(img_row)

                    if iteration is not None and iterator == iteration:
                        break
                    else:
                        iterator+=1
                        if iterator % 1000 == 0: 
                            print(f"Iteration {iterator}") 
                            
                except Exception as e2:
                    problems_rows.append(img_row)
                    print(e2)
                    continue
    except Exception as e:
        print('An exception occurred.', e)
    return problems_rows

In [ ]:
train_missed =  make_dataset_cv("IMG_all", 
                                data, 
                                set_name="train") 